In [1]:
import sys
import os
from copy import deepcopy
sys.path.append("..")
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from webapp.utils.azure_utils import KeyVault, DataLake
import dask.dataframe as dd
import dask.array as da
import river
from data_prep.prep import MeterDataSet

In [2]:
# Connect to Storage Account
vault = KeyVault(keyVaultName = "keyvaultdva2022")
storage_credential = vault.get_secret(secretName = "storagePrimaryKey")
storage = DataLake(account_name = "storageaccountdva", credential = storage_credential)

In [3]:
file_system = "energyhub"
dest_dir = "/data_parq/norm_data"
meters_dir = "/data_parq/meters"
meta_dir = "/data_parq/metadata"
weather_dir = "/data_parq/weather"
bad_building_dir = "/bad_buildings"

In [4]:
meter = "electricity"
metadata_cols = ['building_id', 'site_id','sq_meter', 'primary_space_usage']
weather_cols = ['site_id', 'timestamp', 'air_temperature', 'dew_temperature','wind_direction', 'wind_speed']

## Initiate the Class

In [5]:
electricity = MeterDataSet(meter, metadata_cols, weather_cols)

## Process weather

In [6]:
w_cols = weather_cols[2:]

In [7]:
weather = electricity.fill_weather_na(w_cols, "linear")

In [8]:
len(weather)

331166

In [9]:
weather.columns

Index(['timestamp', 'site_id', 'air_temperature', 'dew_temperature',
       'wind_direction', 'wind_speed'],
      dtype='object')

In [10]:
# weather_df = weather.compute()

In [11]:
# weather_df.isna().sum()*100/len(weather_df)

## Process Meter

In [12]:
meter = electricity.meter

In [13]:
meter.compute().isna().sum()*100/len(meter)

timestamp      0.0
building_id    0.0
electricity    0.0
dtype: float64

## Merge and filter out bad buildings

In [14]:
electricity.merge()

In [15]:
df = electricity.df

In [16]:
# len(df.building_id.unique())
# 1573 unique buildings before filtering bad buildings

1573

,electricity,sq_meter,air_temperature,dew_temperature,wind_direction,wind_speed
npartitions=1,,,,,,
,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...
